tutorial: [Data Camp Jupyter Notebook Tutorial](https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook#gs.ztPiMM0)
* notebook focuses on analysis of data from following command set

# Commands Run To Create DataSets
* all commands run in linux using python 3.5.2
* run times given are for i7-5820K w/ 64 GB RAM
* processing is CPU not memory intensive
* some code could be parallized to reduce run times
>cd /data640g1/data/documents/docs/projects/payton_venusar/VENUSAR_DEV/venusar

## Compute Motif Thresholds
* Motif = TF, language used interchangeably
* 19:11:33 - 17:36:16 = 95 minutes 17 seconds run time
>python3 thresholds.py -fpr 0.001 -m ../../data/HOCOMOCOv10.JASPAR_FORMAT.TF_IDS.txt -o ../../data/HOCOMOCOv10.JASPAR_FORMAT.TF_IDS.fpr_0p001.txt

## Filter Motifs by Gene Expression
* time to run is inconsequential; under a second
>python3 tf_expression.py -i ../../data/FLDL_CCCB_RARE_VARIANTS.MERGED.RNA_DP10.RNA_NODUPS.CHIP_MULTIMARK.SORTED.vcf -o 1 -m ../../data/HOCOMOCOv10.JASPAR_FORMAT.TF_IDS.fpr_0p001.txt -e ../../data/ALL_ARRAYS_NORMALIZED_MAXPROBE_LOG2_COORDS.sorted.txt -mo ../../data/HOCOMOCOv10.JASPAR_FORMAT.TF_IDS.fpr_0p001.txt.bed_reduced.txt

## Modify vcf File to Show Which Motif Matches Changed (reference vs variant)
* runtime ~11 hours with homotypic (without 20170115.02:19:36-20170114.19:10:53 = 7 hours 8 minutes 43 seconds)
>python3 motifs.py -i ../../data/FLDL_CCCB_RARE_VARIANTS.MERGED.RNA_DP10.RNA_NODUPS.CHIP_MULTIMARK.SORTED.vcf -r ../../data/genome_reference/reference_genome_hg19.fa -m ../../data/HOCOMOCOv10.JASPAR_FORMAT.TF_IDS.fpr_0p001.txt.bed_reduced.txt -o ../../data/output.motif.20170114.vcf -fm -fp -ci ../../data/GM12878.ENCODE.ALL_TFS.bed -co ../../data/output.chip_peaks_output.20170114.bed &> ../../data/0_run_logs/20170114_motifs_run_stdout.txt

## z-scores for Chip Peaks in samples with/without variant
* 15:27:32 - 14:51:01 = 36 min 27 seconds run time
* -i is either output of tf_expression if run after motifs, or motifs if it was run later
>python3 activity.py -i ../../data/output.motif.20170114.vcf -a ../../data/QN_FLDL_CCCB_K27AC_PEAKS_SIGNAL.bed -ov ../../data/output.activity.20170114.vcf -ob ../../data/output.activity.20170114.bed -th 2 &> ../../data/0_run_logs/20170114_activity_run_stdout.txt

## z-scores for genes with specified distance from variant
* 15:27:45 - 15:27:33 = 12 seconds run time
>python3 gene_expression.py -i ../../data/output.activity.20170114.vcf -e ../../data/ALL_ARRAYS_NORMALIZED_MAXPROBE_LOG2_COORDS.sorted.txt -ov ../../data/output.gene_expression.20170114.vcf -eth 5 -th 2 &> ../../data/0_run_logs/20170114_gene_expression_run_stdout.txt


In [2]:
import vampire
import motif
import thresholds
import motifs
import activity
import tf_expression
import gene_expression


In [2]:
# plotly by default wants to put everything in the cloud
#    ref: https://plot.ly/python/getting-started/
# must run this code before plotly command calls to avoid account setup request and allow local usage
#    note: plotly.plotly methods are cloud only (ridiculous should be able to use localhost server)
#    instead must use plotly.offline and plotly.iplot
#    iplot is jupyter notebook specific
# Plotly Offline allows you to create graphs offline and save them locally.
#   two methods for plotting offline: plotly.offline.plot() and plotly.offline.iplot().
# ref:https://plot.ly/python/offline/
#
# ref: generally plotly plot ref: https://plot.ly/python/user-guide/

#import plotly.plotly as py # cloud only
#import plotly.graph_objs as go
import plotly
#help(plotly.offline.iplot)
print( plotly.__version__ )

plotly.offline.init_notebook_mode(connected=True)

2.0.0


In [39]:
motif_f_base='../../data/HOCOMOCOv10.JASPAR_FORMAT.TF_IDS.txt'
pc = 0.1
th = 0
bp = [0.25, 0.25, 0.25, 0.25]
motif_set_base = motif.get_motifs(motif_f_base, pc, th, bp)

In [40]:
motif_f_expressed='../../data/HOCOMOCOv10.JASPAR_FORMAT.TF_IDS.fpr_0p001.txt.bed_reduced.txt'
motif_set_expressed = motif.get_motifs(motif_f_expressed, pc, th, bp)


In [41]:
print( "length " + format(len(motif_set_expressed.motifs)) )

print( "length " + format(motif_set_expressed.length()) )

length 603
length 603


In [58]:
motif_lengths_base = motif_set_base.element_positions_list(False)
motif_lengths_expressed = motif_set_expressed.element_positions_list(False)
print( "motif_lengths_base type: " + format(type(motif_lengths_base)) + " has " + format(len(motif_lengths_base)) + " elements.")
print( "motif_lengths_expressed type: " + format(type(motif_lengths_expressed)) + " has " + format(len(motif_lengths_expressed)) + " elements.")
print( "motif_lengths_expressed has " + format(len(motif_lengths_base) - len(motif_lengths_expressed)) + " fewer elements.")

motif_lengths_base type: <class 'list'> has 641 elements.
motif_lengths_expressed type: <class 'list'> has 603 elements.
motif_lengths_expressed has 38 fewer elements.


In [46]:

# ref: https://plot.ly/python/histograms/

data = [
    plotly.graph_objs.Histogram(
        x=motif_lengths_base
    )
]

layout = plotly.graph_objs.Layout(
    title='TF Length Histogram after running tf_expression.py',
    xaxis=dict(
        title='TF Length',
        ticklen=5,
        zeroline=False,
        gridwidth=2,
    ),
    yaxis={'title':'Number of TF with Length'}
    )
#plotly.offline.iplot(data)  # basic plot
go_figure = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(go_figure)

In [62]:
# repeating graph setup using reproducible code
# ref: https://plot.ly/python/histograms/

# -- setup the layout information to reuse
xaxis_template = {
        'title':'TF Length',
        'ticklen':5,
        'zeroline':False,
        'gridwidth':2,
    }
yaxis_template ={'title':'Number of TF with Length'}

# -- plot 1
hist_all_TF = plotly.graph_objs.Histogram(
        name='All TF',
        x=motif_lengths_base,
        opacity=0.75
    )
layout = plotly.graph_objs.Layout(
    title='TF Length Histogram (all TF)',
    xaxis=xaxis_template,
    yaxis=yaxis_template
    )

go_figure = plotly.graph_objs.Figure(data=[hist_all_TF], layout=layout)
plotly.offline.iplot(go_figure)

# -- plot 2
hist_expressed_TF = plotly.graph_objs.Histogram(
        name='Expressed TF',
        x=motif_lengths_expressed,
        opacity=0.75
    )
layout = plotly.graph_objs.Layout(
    title='TF Length Histogram after running tf_expression.py',
    xaxis=xaxis_template,
    yaxis=yaxis_template
    )

go_figure = plotly.graph_objs.Figure(data=[hist_expressed_TF], layout=layout)
plotly.offline.iplot(go_figure)

# -- plot overlay
layout = plotly.graph_objs.Layout(
    title='TF Length Histogram comparison tf_expression.py dropped',
    xaxis=xaxis_template,
    yaxis=yaxis_template,
    barmode='overlay'
    )
go_figure = plotly.graph_objs.Figure(data=[hist_all_TF, hist_expressed_TF], layout=layout)
plotly.offline.iplot(go_figure)



In [61]:
help(go_figure.layout)
go_figure.layout

Help on PlotlyDict in module plotly.graph_objs.graph_objs object:

class PlotlyDict(builtins.dict, PlotlyBase)
 |  Base class for dict-like Plotly objects.
 |  
 |  Method resolution order:
 |      PlotlyDict
 |      builtins.dict
 |      PlotlyBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __copy__(self)
 |  
 |  __deepcopy__(self, memodict={})
 |  
 |  __dir__(self)
 |      Dynamically return the existing and possible attributes.
 |  
 |  __getattr__(self, key)
 |      Python only calls this when key is missing!
 |  
 |  __getitem__(self, key)
 |      Calls __missing__ when key is not found. May mutate object.
 |  
 |  __init__(self, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __missing__(self, key)
 |      Mimics defaultdict. This is called from __getitem__ when key DNE.
 |  
 |  __setattr__(self, key, value)
 |      Maps __setattr__ onto __setitem__
 |  
 |  __setitem__(self, key, value, _raise=True)
 |     

{'barmode': 'overlay',
 'title': 'TF Length Histogram comparison tf_expression.py dropped',
 'xaxis': {'gridwidth': 2,
  'ticklen': 5,
  'title': 'TF Length',
  'zeroline': False},
 'yaxis': {'title': 'Number of TF with Length'}}

In [ ]:
# next create a reader that exports motifs=TF, samples, and variant information from the vcf file
#   as a python structure
#
#   get vcf reader for variant elements from motifs.py
#   get vcf reader for samples from gene_expression.py
#
# then do similar and expanded analysis looking at which motifs were selected/exported by variant
# could also group by variant type
#   a->t, etc
#


# Expression Plots

## z-scores for genes with specified distance from variant

* run-time seconds
* operates against gene_expression.py output vcf file
>python3 summarize.py -i ../../data/output.gene_expression.20170114.vcf -o ../../data/output.gene_expression.20170114.table




In [4]:
# create a pandas data frame of the data table
import pandas as pandas
vcf_table_file = '../../data/output.gene_expression.20170114.table_full.txt'
vcf_table = pandas.read_csv(vcf_table_file, sep='\t')

In [5]:
# confirm type
type(vcf_table)

pandas.core.frame.DataFrame

In [6]:
vcf_table.sort_values('DISTANCE', ascending=False)

,CHR,POS,REF,ALT,MOTIF,LOG2_VAR-REF_SCORE,SAMPLE,LOCIID,ACT_ZSCORE,GENE,EXP_ZSCORE,DISTANCE
40525,chr11,102217809,C,CCT,SCRT2,34.0930,CB021314,7230,2.4329,BIRC3,-2.5774,34.276736
40520,chr11,102217809,C,CCT,SCRT2,34.0930,FL313,7230,3.1783,TMEM123,-0.8938,34.252491
40521,chr11,102217809,C,CCT,SCRT2,34.0930,FL313,7230,3.1783,BIRC2,0.8854,34.252273
40519,chr11,102217809,C,CCT,SCRT2,34.0930,FL313,7230,3.1783,BIRC3,0.7117,34.248223
40527,chr11,102217809,C,CCT,SCRT2,34.0930,CB021314,7230,2.4329,BIRC2,-1.5596,34.215260
40526,chr11,102217809,C,CCT,SCRT2,34.0930,CB021314,7230,2.4329,TMEM123,0.9745,34.193586
40523,chr11,102217809,C,CCT,SCRT2,34.0930,DL551,7230,-1.1479,TMEM123,-1.5653,34.148214
40522,chr11,102217809,C,CCT,SCRT2,34.0930,DL551,7230,-1.1479,BIRC3,-1.2232,34.134243
40524,chr11,102217809,C,CCT,SCRT2,34.0930,DL551,7230,-1.1479,BIRC2,-0.8806,34.123684
40530,chr11,102217809,C,CCT,SCRT2,34.0930,DL3A538,7230,1.3188,BIRC2,0.3539,34.120333


In [7]:
vcf_table.size

2349696

In [8]:
vcf_table[vcf_table.DISTANCE > 16].size

46080

In [9]:
vcf_table.plot.hist()

In [10]:
# XXX: This doesn't work yet; fails creating new column
# ref: https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf
# ref: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html
import numpy
motif_score_scale = max(vcf_table['LOG2_VAR-REF_SCORE'] ** 2)
act_score_scale = max(vcf_table.ACT_ZSCORE ** 2)
gene_score_scale = max(vcf_table.EXP_ZSCORE ** 2)

print('Score Scaling Factors:\n\tMotif = \t', motif_score_scale, '\n\tActivity = \t', act_score_scale, '\n\tGene = \t\t', gene_score_scale ) 
#vcf_table['distance_w'] = vcf_table.apply( numpy.sqrt, ((vcf_table['LOG2_VAR-REF_SCORE'] ** 2) / motif_score_scale) + 
#                               ((vcf_table.ACT_ZSCORE ** 2) / act_score_scale ) + 
#                               ((vcf_table.EXP_ZSCORE ** 2) / gene_score_scale ) )
vcf_table['distance_w'] = numpy.sqrt( ((vcf_table['LOG2_VAR-REF_SCORE'] ** 2) / motif_score_scale) + ((vcf_table.ACT_ZSCORE ** 2) / act_score_scale ) + ((vcf_table.EXP_ZSCORE ** 2) / gene_score_scale ) )
vcf_table.sort_values('distance_w', ascending=False)

Score Scaling Factors:
	Motif = 	 1162.332649 
	Activity = 	 198.51964609 
	Gene = 		 67.24328004


,CHR,POS,REF,ALT,MOTIF,LOG2_VAR-REF_SCORE,SAMPLE,LOCIID,ACT_ZSCORE,GENE,EXP_ZSCORE,DISTANCE,distance_w
167811,chr6,24990939,GAAAAAA,"GAAAAAAA,GAAAAAAAA",FOXL1,31.3370,DL551,33638,-1.1346,FAM65B,-7.1462,32.161515,1.269172
168111,chr6,24990939,GAAAAAA,"GAAAAAAA,GAAAAAAAA",CPEB1,27.6798,DL551,33638,-1.1346,FAM65B,-7.1462,28.609908,1.193778
10824,chr1,178698151,A,G,KLF14,14.4409,DL551,2884,-1.1481,RALGPS2,-8.2002,16.646351,1.089061
135786,chr2,198093455,G,A,FOXD2,13.2244,DL551,22922,0.3094,ANKRD44,-8.1858,15.555957,1.071183
149807,chr3,130614653,C,A,CEBPA,12.2880,DL135,28070,14.0897,ATP2C1,-1.0391,18.724164,1.070497
10692,chr1,178698151,A,G,ELF1,-12.2764,DL551,2884,-1.1481,RALGPS2,-8.2002,14.807816,1.065974
40525,chr11,102217809,C,CCT,SCRT2,34.0930,CB021314,7230,2.4329,BIRC3,-2.5774,34.276736,1.062359
10776,chr1,178698151,A,G,ETV1,-11.6755,DL551,2884,-1.1481,RALGPS2,-8.2002,14.313585,1.060150
149957,chr3,130614653,C,A,BCL6B,9.3465,DL135,28070,14.0897,ATP2C1,-1.0391,16.939789,1.044612
10800,chr1,178698151,A,G,ETV6,-8.9482,DL551,2884,-1.1481,RALGPS2,-8.2002,12.191460,1.037076


In [11]:
data = [
    plotly.graph_objs.Histogram(
        x=vcf_table.distance_w
    )
]

layout = plotly.graph_objs.Layout(
    title='Scaled Score Distance Histogram',
    xaxis=dict(
        title='Scaled Score',
        ticklen=5,
        zeroline=False,
        gridwidth=2,
    ),
    yaxis={'title':'Number of Samples with Distance Score'}
    )
#plotly.offline.iplot(data)  # basic plot
go_figure = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(go_figure)

In [12]:
vcf_tabletopx = vcf_table[vcf_table.distance_w > .8]
vcf_tabletopx.size

8710

In [14]:
trace1 = plotly.graph_objs.Scatter3d(
    name="Distances",
    x=vcf_tabletopx['LOG2_VAR-REF_SCORE'],
    y=vcf_tabletopx.ACT_ZSCORE,
    z=vcf_tabletopx.EXP_ZSCORE,
    mode='markers',
    marker=dict(
        size=4,                 # Using gene expression as color scale values for now.
        color=vcf_tabletopx.EXP_ZSCORE,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace1]
layout = plotly.graph_objs.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    title='Distances from Average for Individual Variant Events',
    scene=dict(
        xaxis=dict(
            title='log2 Var/Ref Motif Log-Odds Ratio Difference',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='Enhancer Activity z-Score',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        ),
        zaxis=dict(
            title='Gene Expression z-Score',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
)

go_figure = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(go_figure)

In [21]:
# ref: http://stackoverflow.com/questions/21800169/python-pandas-get-index-of-rows-which-column-matches-certain-value
vcf_sub = vcf_table[vcf_table.MOTIF == 'RFX5']
print( 'original table size:', vcf_table.size )
print( 'reduced RFX5 set:', vcf_sub.size )
vcf_sub[vcf_sub['LOG2_VAR-REF_SCORE'] == (5.4636-8.4653)]


original table size: 2545504
reduced RFX5 set: 8177


,CHR,POS,REF,ALT,MOTIF,LOG2_VAR-REF_SCORE,SAMPLE,LOCIID,ACT_ZSCORE,GENE,EXP_ZSCORE,DISTANCE,distance_w
